#  How to quickly and easily import flat maps into pycortex

Easy (and hacky!!) way to import freesurfer ROIs into pycortex

BEFORE YOU DO THIS MAKE SURE TO BACK UP ANY KIND OF PYCORTEX STUFF YOU DON'T WANT OVERWRITTEN!!!

Why would you do this? Because pycortex is beautiful, but you are lazy... 

It is really nice to be able to toggle the ROIs on and off in pycortex viewer; but to do this you have to have them defined on the svg. To define them on the SVG, you need to draw them in inkscape. But maybe you already used freesurfer, and don't want to... whatever

This helps with that...

It also works with anything defined as a boolean array. 

Also for plotting flatmaps in a pycortex like way but with more flexibility with matplotlib could be useful


In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
opj = os.path.join

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# FOR MY OWN TESTING OF THIS EXAMPLE FOLDER
# -> Freesurfer directory
sub = 'sub-02'
fs_dir = '/data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/freesurfer'
ctx_dir = '/data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/cortex'
# -> load design matrix, time series and prfpy parameters from one of my projects
# 
from pfa_scripts.load_saved_info import *
prfpy_numpy_array = load_data_prf('sub-02', 'AS0', 'gauss')['AS0']['gauss']
time_series = load_data_tc(sub=sub, ses='ses-1', task_list='AS0', )['AS0']
design_matrix = get_design_matrix_npy(['AS0'])['AS0'][:,:,5:]         

# Create prfpy stimulus & model
assert design_matrix.shape[-1]==time_series.shape[-1]
prf_stim = PRFStimulus2D(
    screen_size_cm=39.3,                                    # height of the screen (i.e., the diameter of the stimulated region)    
    screen_distance_cm=196,                                 # Distance of screen to eye
    design_matrix=design_matrix,                            # dm (npix x npix x time_points)
    TR=1.5,                                                 # TR
    )   
prfpy_model = Iso2DGaussianModel(stimulus=prf_stim)
from dag_prf_utils.prfpy_ts_plotter import TSPlotter
# Now make the "ts plotter": which will make plotting our prf information very easy
prfs  = TSPlotter(
    prf_params=prfpy_numpy_array,
    model='gauss',
    prfpy_model=prfpy_model,
    real_ts=time_series
)

# If you don't want to work with prfpy directly -> just load your own version for some of these
# ** Control visibility of data ** [using mask, or alpha, or both...] 
# If you don't want to show the values of every point (for example because it is outside the visual cortex). You may want to hide it. 
# If you are plotting PRFs, then you may want to hide the bad fits. So you can create a mask for where the rsq<threshold (e.g., 0.1)
# data_mask: what to show (TRUE), what to hide (FALSE)
# -> should boolean 1D np.ndarray, where the length = number of vertices in subject surface
# -> if unspecified, all surface functions assume TRUE for all voxels
# data_alpha: transparency (invisible = 0), (opaque = 1).
# Often you will do this based on the rsquared of your model

polar_angle_data = prfs.pd_params['pol'].to_numpy()
eccentricity_data = prfs.pd_params['ecc'].to_numpy()
data_mask = prfs.return_vx_mask(th={'min-rsq':.1, 'max-ecc':5})
data_rsq = prfs.pd_params['rsq'].to_numpy()
data_alpha = data_rsq.copy()


The nilearn.glm module is experimental. It may change in any future release of Nilearn.


/data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/prf_no_hrf/sub-02/ses-1/sub-02_model-gauss_roi-all_task-AS0-fits_stage-iter_constr-tc_desc-prf_params.pkl
prf_params.shape[-1]=8
include hrf = True
include rsq = True


In [11]:
from dag_prf_utils.pycortex import set_ctx_path,PyctxMaker

# Make sure pycortex is pointing to correct folder
# set_ctx_path('/path/to/pycortex/files')

pm = PyctxMaker(
    sub = sub,
    fs_dir = fs_dir, 
    ctx_path=ctx_dir,
    )


Using fs dir = /data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/freesurfer
/data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/cortex
default
/data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/cortex/sub-02
b'created by abdirashid on Tue Nov  9 18:55:52 2021\n'
b'created by abdirashid on Tue Nov  9 20:57:32 2021\n'
b'created by abdirashid on Tue Nov  9 18:55:52 2021\n'
b'created by abdirashid on Tue Nov  9 22:17:22 2021\n'


## quick flatmaps
Already got flatmaps? then skip this stage

Pycortex needs flatmaps. In the form of flat_*h.gii in the pycortex folder for that subject. If you are very lazy (like me) and don't want to go through the trouble of flattening - i offer a couple of DODGY ways  to flatten the surface. 

It involves selecting some of the vertices to focus on. Then applying some kind of flattening algorithm. I then save the stuff into the pycortex folder

In [12]:
import inspect
from dag_prf_utils.utils import dag_load_roi
# Lets pick some vertices to focus on:
# I have drawn a bunch of ROIs in the freesurfer folder
# all called 'custom', so I'll load them
# But you can make this selection anyway you want
centre_bool = dag_load_roi(
    sub=sub,
    roi='v1custom', 
    fs_dir=fs_dir,
    combine_matches=True,  
)

print(f'Number of vertices: {centre_bool.sum()}')
# Boolean array of vertices (in both hemispheres)

Number of vertices: 4905


Make the flatmap 

Basically take the focussing on the coordinates, we can do some flattening. This can be primitively done using the freesurfer projection to a sphere (and just using latitude and longitude). Or you can use the "igl" package which does some clever as-rigid-as-possible transformations. IGL is better, but can fail randomly during the making - so you may have to rerun until you are happy

In [ ]:
print(inspect.getsource(pm.make_flat_map))

In [13]:
if True: # DONT DO IT THE SECOND TIME
    pm.mesh_info['occ'] = None
    pm.make_flat_map_CUSTOM(
        centre_bool=centre_bool,
        method='latlon', # 'igl' or 'latlon'
        hemi_project='inflated', # Starting point for flattening (e.g., inflated, pial, sphere)
        morph=10, # Can expand or shrink the selection on the mesh (dilation/erosion)
        flat_name='flat',
        # cut_box=True, 
    )
    # Morphing is useful to make sure the selection is not too small

centering!
Faces with missing vx: 271564
Faces with long edges: 1197
0.036371507448175036
saving to /data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/cortex/sub-02/surfaces/flat_lh.gii
centering!
Faces with missing vx: 273562
Faces with long edges: 1162
0.03328833636061657
saving to /data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/cortex/sub-02/surfaces/flat_rh.gii
saving to /data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/cortex/sub-02/custom_flat_files/flat.npz
flat
All done! Remember to restart the kernel before trying to make the new overlay


In [14]:
pm.make_svg()

You need to have already made the flatmap (.gii) files
(282402, 2)
0.0 835.6851196289062
0.0 1024.0


before we go any further - lets check that it turned out how we would like

## Add the ROIs to pycortex SVG (Kill kernel and rerun first!)
To add ROIs we 

[1] Load the boolean array from freesurfer

[2] Find the vertices on the edge of ROIs

[3] Find the order for these vertices 

[4] Draw them onto our SVG file


Again I've written this so you can do it just using this line of code

In [15]:
roi_list = ['v1', ]  # roi names in you're freesurfer label folder
pm.add_rois_to_svg(roi_list)

In [10]:
pm.quick_show()

Just using undersurface file..


OSError: 